In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [2]:
# Define CNN model
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    
    # First Convolutional Block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Second Convolutional Block
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Third Convolutional Block
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Prevent overfitting
    
    # Output Layer (Softmax for multi-class classification)
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model





: 

In [ ]:
input_shape = (128, 128, 3) 
num_classes = 4  

model = create_cnn_model(input_shape, num_classes)

# Model Summary
model.summary()

# Assuming you have your data in train_images, train_labels, test_images, and test_labels

# Convert labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels, num_classes)
test_labels_onehot = to_categorical(test_labels, num_classes)

# Data Augmentation to avoid overfitting (optional)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model
batch_size = 32
epochs = 25

history = model.fit(
    datagen.flow(train_images, train_labels_onehot, batch_size=batch_size),
    validation_data=(test_images, test_labels_onehot),
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs
)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_images, test_labels_onehot)
print(f'Test Accuracy: {test_acc:.2f}')
